In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

from PIL import Image
import os
import numpy as np
from collections import defaultdict


images_dir="/content/drive/MyDrive/jsontopng"
os.chdir(images_dir)

def reclassify_pixels(image, tag):
    """
    Reclassify pixel values in the image based on the tag.
    - "Airplane": 255 -> 1, everything else -> 0
    - "Car": 255 -> 2, everything else -> 0
    """
    # Convert the image to a numpy array
    img_array = np.array(image)

    # Reclassify based on the tag
    if tag == "Potato":
        img_array = np.where(img_array == 255, 255, 0)
    elif tag == "Paddy":
        img_array = np.where(img_array == 255, 85, 0)
    elif tag == "Maize":
        img_array = np.where(img_array == 255, 170, 0)

    return img_array

def combine_images(image_list):
    """
    Combine images by taking the maximum pixel value at each position.
    """
    # Start with a blank canvas (all zeros) based on the shape of the first image
    combined_array = np.zeros_like(image_list[0])

    # Iterate through all images and combine them
    for img_array in image_list:
        combined_array = np.maximum(combined_array, img_array)

    return combined_array

def process_images():
    # Group files by source image name
    grouped_files = defaultdict(list)
    for file_name in os.listdir():
        if file_name.endswith('.png'):
            # Extract the source image name and tag
            parts = file_name.split('-by-')
            source_name = parts[0]
            tag = parts[-1].split('-')[2]

            grouped_files[source_name].append((file_name, tag))

    # Process each group of images
    for source_name, files in grouped_files.items():
        reclassified_images = []

        for file_name, tag in files:
            # Open the image
            with Image.open(file_name) as img:
                # Reclassify pixel values based on the tag
                reclassified_img = reclassify_pixels(img, tag)
                reclassified_images.append(reclassified_img)

        # Combine all reclassified images for this source
        combined_image = combine_images(reclassified_images)

        # Save the combined image
        combined_image_pil = Image.fromarray(combined_image.astype(np.uint8))
        if not os.path.exists('combinedoutput'):
          os.mkdir('combinedoutput')
        output_location = os.path.join(os.getcwd(),'combinedoutput',f"{source_name}-combined.png")
        combined_image_pil.save(output_location)

# Run the process
process_images()


In [ ]:
csv_path='/content/drive/MyDrive/Jsonn_and_csv/project-24-at-2024-12-21-15-01-93d36c5d.csv'
import pandas as pd
df= pd.read_csv(csv_path)
df.head()


,annotation_id,annotator,created_at,id,image,label,lead_time,updated_at
0,2331,1,2024-12-02T10:19:43.143829Z,3078,/data/upload/24/2508006e-Banepa_Tunnel_Feasibi...,"[{""points"":[[1.773937724350947,99.582602888388...",1093.201,2024-12-04T08:37:51.486782Z
1,2332,1,2024-12-02T10:20:23.419938Z,3079,/data/upload/24/a0dca484-Banepa_Tunnel_Feasibi...,"[{""points"":[[0.2777777777777778,99.58333333333...",304.213,2024-12-02T11:37:22.435763Z
2,2333,1,2024-12-02T11:40:52.457489Z,3080,/data/upload/24/ccf67c80-Banepa_Tunnel_Feasibi...,"[{""points"":[[0.2777777777777778,99.44444444444...",258.642,2024-12-02T11:42:09.407752Z
3,2334,1,2024-12-02T11:45:26.477324Z,3081,/data/upload/24/b615433e-Banepa_Tunnel_Feasibi...,"[{""points"":[[0.5555555555555556,87.5],[0.41666...",200.882,2024-12-02T11:45:40.208000Z
4,2335,1,2024-12-02T11:50:22.634410Z,3082,/data/upload/24/c1e2d0e3-Banepa_Tunnel_Feasibi...,"[{""points"":[[0.2777777777777778,93.88888888888...",334.021,2024-12-02T11:51:17.174038Z


In [ ]:
import re

def extract_annotation_id(filename):
    # Use regular expression to find the annotation_id
    match = re.search(r'task-(\d+)-annotation', filename)

    if match:
        # Return the extracted annotation_id
        return int(match.group(1))
    else:
        # Handle cases where the pattern doesn't match
        raise ValueError(f"Filename {filename} does not match the expected pattern.")

# Example usage
filename = "task-2046-annotation-1353-combined.png"
annotation_id = extract_annotation_id(filename)

print(annotation_id)  # Output will be 2046


2046


In [ ]:
import pandas as pd
import os




# Function to map annotation_id to base name of image
def map_annotation_to_base(df):
    # Extract the base name of the file from the image column
    df['base_name'] = df['image'].apply(lambda x: os.path.basename(x).split('-')[1])

    # Create a dictionary to map annotation_id to base name
    mapping_dict = df.set_index('id')['base_name'].to_dict()

    return mapping_dict

# Using the function
annotation_to_base_mapping = map_annotation_to_base(df)

# Output the mapping
print(annotation_to_base_mapping)


{3078: 'Banepa_Tunnel_Feasibility_Survey_transparent_mosaic_group138030.PNG', 3079: 'Banepa_Tunnel_Feasibility_Survey_transparent_mosaic_group138036.PNG', 3080: 'Banepa_Tunnel_Feasibility_Survey_transparent_mosaic_group138037.PNG', 3081: 'Banepa_Tunnel_Feasibility_Survey_transparent_mosaic_group138038.PNG', 3082: 'Banepa_Tunnel_Feasibility_Survey_transparent_mosaic_group138039.PNG', 3083: 'Banepa_Tunnel_Feasibility_Survey_transparent_mosaic_group138040.PNG', 3084: 'Banepa_Tunnel_Feasibility_Survey_transparent_mosaic_group138041.PNG', 3085: 'Banepa_Tunnel_Feasibility_Survey_transparent_mosaic_group138042.PNG', 3086: 'Banepa_Tunnel_Feasibility_Survey_transparent_mosaic_group138043.PNG', 3087: 'Banepa_Tunnel_Feasibility_Survey_transparent_mosaic_group138044.PNG', 3088: 'Banepa_Tunnel_Feasibility_Survey_transparent_mosaic_group138045.PNG', 3089: 'Banepa_Tunnel_Feasibility_Survey_transparent_mosaic_group138046.PNG', 3090: 'Banepa_Tunnel_Feasibility_Survey_transparent_mosaic_group138397.PNG'

In [ ]:
def rename_mask(mask_name):
  #mask_name='/content/drive/MyDrive/Combiningmask/Mask/output/task-1977-annotation-1285-combined.png'
  mask_basename=os.path.basename(mask_name)
  mask_dirname=os.path.dirname(mask_name)
  annotation_id = extract_annotation_id(mask_basename)
  output_maskname=annotation_to_base_mapping[annotation_id].replace('.PNG','_mask.PNG')
  full_outputname=os.path.join(mask_dirname,output_maskname)
  os.rename(mask_name,full_outputname)
  print(mask_name,' : ',full_outputname)


In [ ]:
mask_dir='/content/drive/MyDrive/jsontopng/combinedoutput'
images=os.listdir(mask_dir)
for each in images:
  rename_mask(
      os.path.join(mask_dir,each)
  )

KeyError: 2631